In [11]:
from db import call_ignition, call_db_json, insert_many_with_df, update_ignition
from queries import hk_flag, get_set_flag
from util_func import get_mondays, BlockKanbanStorage, BlockCoverage
from datetime import date, timedelta
import pandas as pd
import numpy as np
import bisect

block_kanban = BlockKanbanStorage()
block_coverage = BlockCoverage()

In [55]:
def block_demand(force_update=False):
    today = date.today()
    query_day = 168 - timedelta(today.weekday()).days
    sql = f"""
    SELECT product_family, product_model, item_description, product_variant, order_scheduled_due, printed_due, SUM(order_quantity) AS qty, ARRAY_AGG(manufacturing_orders.order_number) as MOS, item_number, reference_number, facility_id
        FROM manufacturing_orders
        INNER join manufacturing_order_processes on manufacturing_orders.order_number = manufacturing_order_processes.order_number
        WHERE order_status ='10'
            AND order_release_code = 5
            AND facility_id in  ('A0010', 'A0045', 'E0010', 'J0005')
            AND product_model IS NOT NULL
            AND reference_number IS NOT NULL
            AND item_description like '%BLOCK%'
            AND order_scheduled_due <= CURRENT_DATE + {query_day}
        GROUP BY (product_family, product_model, product_variant, item_description, item_number, printed_due, reference_number, order_scheduled_due, printed_due, facility_id)    
    """
    return call_db_json(sql)


def check_date_group(dt, mondays):
    index = bisect.bisect_right(mondays, dt)
    if index == len(mondays):
        return 25
    else:
        return index

# grouped_df
def get_name(row):
    adder = "X" if row.product_family == "HSR" else ""
    return (
        row["product_family"]
        + row["product_model"]
        + adder
        + row["product_variant"]
        + row["is_M"]
    )


def get_block_process():
    block_proc_sql = """
    SELECT
        order_number,
        product_family,
        product_model,
        product_variant,
        item_description,    
        order_quantity,
        printed_due,
        item_number,
        (
        SELECT
            string_agg(facility_id, ' ')
        FROM manufacturing_order_processes
        WHERE manufacturing_order_processes.order_number = manufacturing_orders.order_number
        ) facility,
        (SELECT
            facility_id
        FROM manufacturing_order_processes
        WHERE operation_status != '40'
            AND manufacturing_order_processes.order_number = manufacturing_orders.order_number
        ORDER BY operation_sequence
        LIMIT 1) facility_id,
        reference_number,
        order_scheduled_due,
        (SELECT
            time_in
        FROM manufacturing_order_logs
        WHERE manufacturing_order_logs.order_number = manufacturing_orders.order_number
        ORDER BY time_in DESC
        LIMIT 1) time_out
        FROM manufacturing_orders
        WHERE manufacturing_orders.order_status ='40'
            AND manufacturing_orders.order_release_code = 5
            AND product_model IS NOT NULL
            AND product_family IS NOT NULL
            AND product_variant IS NOT NULL
            AND item_description NOT LIKE '%+%'
            AND (reference_number LIKE 'HK%' or reference_number IN ('GY2', 'GY3', 'GY4'))
            AND product_block_count > 0
            AND product_family != 'TS'
        ORDER BY order_number
    """
    proc_db = call_db_json(block_proc_sql)
    pdf = pd.DataFrame(proc_db)
    pdf["is_M"] = pdf["item_description"].apply(lambda x: "_M" if " M " in x else "")
    pdf["item_type"] = pdf.apply(get_name, axis=1)
    pdf["gy"] = pdf["reference_number"].apply(lambda x: "HK" if "HK" in x else "GY")
    rename = {"order_quantity": "qty", "order_number": "mos"}
    pdf = pdf.rename(columns=rename)
    p_groupby = pdf.groupby(["item_type", "facility_id", "gy"])
    p_grouped_df = p_groupby["qty"].sum()
    p_grouped_df = p_grouped_df.reset_index().set_index("item_type")
    p_grouped_df = p_grouped_df.pivot_table(
        index=["item_type", "facility_id"], columns="gy", values="qty", aggfunc="sum"
    ).reset_index()
    p_grouped_df.columns = ["item_type", "facility_id", "gy_qty", "hk_qty"]
    p_grouped_df["gy_qty"].fillna(0, inplace=True)
    p_grouped_df["hk_qty"].fillna(0, inplace=True)
    pdf = pdf[
        [
            "item_description",
            "order_scheduled_due",
            "qty",
            "reference_number",
            "mos",
            "item_type",
            "facility_id",
            "gy",
        ]
    ]
    return p_grouped_df, pdf


def get_block_stock():
    block_stock_sql = """
        SELECT item_number, item_description, product_family, product_model, product_variant, product_block_count, warehouse_location, quantity, virtual_location
            FROM current_wip_inventory
            WHERE block=true
                AND product_variant IS NOT NULL
                AND warehouse_location != 'INTRAN'
        """
    s_df = pd.DataFrame(call_db_json(block_stock_sql))
    s_df.head()
    s_df["is_M"] = s_df["item_description"].apply(lambda x: "_M" if " M " in x else "")
    s_df["item_type"] = s_df.apply(get_name, axis=1)
    s_df["virtual_location"] = s_df["virtual_location"].apply(
        lambda x: "GY1" if x == "BLOCK" else x
    )

    s_df["warehouse_location"] = s_df["warehouse_location"].apply(
        lambda x: "SET" if "SET" in x else ""
    )

    # s_df.loc[s_df['item_description'].str.contains('HOLED BLOCK'), 'virtual_location'] = 'GY2.5'
    s_groupby = s_df.groupby(["item_type", "virtual_location", "warehouse_location"])
    qty = s_groupby["quantity"].sum().reset_index()
    s_df = qty.pivot_table(
        index=["item_type"],
        columns=["warehouse_location", "virtual_location"],
        values="quantity",
    )
    s_df = s_df.fillna(0).reset_index()
    s_df.columns = ["item_type", "GY1", "GY2", "GY3", "GY4_gk", "GY4_set"]
    s_df[["GY4_gk", "GY4_set"]] = s_df[["GY4_gk", "GY4_set"]].astype(int)

    def gy4_concat(row):
        qty = row["GY4_gk"] + row["GY4_set"]
        combined_string = str(row["GY4_gk"]) + "/" + str(row["GY4_set"])
        return np.array([qty, combined_string])

    s_df["GY4"] = s_df.apply(gy4_concat, axis=1)
    s_df = s_df.drop(columns=["GY4_gk", "GY4_set"])
    s_df = pd.melt(
        s_df, id_vars="item_type", var_name="facility_id", value_name="order_quantity"
    )
    return s_df


def gy_checker(facility_id):
    if facility_id == "J0005":
        return "GY4"
    elif facility_id == "E0010":
        return "GY3"
    else:
        return "GY2"


def get_block_demand():
    mondays = get_mondays(23)
    mondays = [i.date() for i in mondays]

    demand = pd.DataFrame(block_demand())
    try:
        demand["hk_flag"] = demand["reference_number"].apply(lambda x: hk_flag.flag[x])
    except:
        get_set_flag(True)
        demand["hk_flag"] = demand["reference_number"].apply(lambda x: hk_flag.flag[x])
    demand["order_scheduled_due"] = pd.to_datetime(demand["order_scheduled_due"])

    demand["week"] = demand["order_scheduled_due"].apply(
        check_date_group, args=[mondays]
    )
    demand["is_M"] = demand["item_description"].apply(
        lambda x: "_M" if " M " in x else ""
    )
    demand = demand.rename(columns={"facility_id": "gy"})

    demand["gy"] = demand["gy"].apply(gy_checker)
    demand["item_type"] = demand.apply(get_name, axis=1)
    d_groupby = demand.groupby(
        ["product_family", "product_model", "product_variant", "is_M", "week", "gy"]
    )

    qty = d_groupby["qty"].sum()
    demand_grouped = qty.reset_index()
    demand_grouped["item_type"] = demand_grouped.apply(get_name, axis=1)
    demand_grouped = demand_grouped.drop(
        columns=["product_family", "product_model", "product_variant", "is_M"]
    )
    demand_grouped = demand_grouped.pivot_table(
        index=["item_type", "week"], columns="gy", values="qty", aggfunc="sum"
    ).reset_index()
    demand_grouped.columns = ["item_type", "week", "gy2_qty", "gy3_qty", "gy4_qty"]
    # demand_grouped[["gy2_qty", "gy3_qty","gy4_qty"]].fillna(0, inplace=True)
    demand_grouped["gy2_qty"].fillna(0, inplace=True)
    demand_grouped["gy3_qty"].fillna(0, inplace=True)
    demand_grouped["gy4_qty"].fillna(0, inplace=True)
    # demand_grouped.fillna(0, inplace=True)
    demand = demand[
        [
            "item_description",
            "order_scheduled_due",
            "qty",
            "reference_number",
            "mos",
            "item_type",
            "week",
            "hk_flag",
            "gy",
        ]
    ].copy()
    demand["block_covered"] = False
    demand = demand.sort_values("order_scheduled_due")
    return demand_grouped, demand, mondays


def block_processing_write_databse(processing, proc_stoc_due_map):
    processing = processing.merge(
        proc_stoc_due_map[["item_type", "facility_id", "due"]],
        on=["item_type", "facility_id"],
        how="left",
    )
    processing.loc[
        processing["order_scheduled_due"].isnull(), "order_scheduled_due"
    ] = processing["due"]
    processing = processing.fillna("")
    processing["order_scheduled_due"] = processing["order_scheduled_due"].apply(
        lambda x: x.strftime("%Y-%m-%d") if x != "" else None
    )
    processing = (
        processing.drop(columns=["gy", "due"])
        .rename(columns={"mos": "mo"})
        .fillna("NO DUE")
    )
    processing = processing.groupby(["facility_id"]).apply(
        lambda x: x.sort_values("order_scheduled_due")
    )
    processing.reset_index(drop=True, inplace=True)
    processing["facility_priority"] = processing.groupby(["facility_id"]).cumcount() + 1
    table = "ignition.initial_release.block_priority"
    delete_sql = f"TRUNCATE {table}; DELETE FROM {table}"
    update_ignition(delete_sql)
    insert_many_with_df("ignition", processing, "initial_release.block_priority")


def generate_block_kanban():
    facility_list = [
        "A0020",
        "A0030",
        "GY2",
        "A0050",
        "GY2.5",
        "A0070",
        "GY3",
        "E0020",
        "F0020",
        "E0030",
        "E0050",
        "GY4",
    ]
    facility_dict = {}
    for i, v in enumerate(facility_list):
        facility_dict[v] = i
    demand_len = 25
    proc_stoc_len = len(facility_list)
    demand_grouped, demand, mondays = get_block_demand()
    processing_grouped, processing = get_block_process()
    stock_df = get_block_stock()
    proc_stoc = pd.concat([processing_grouped, stock_df]).reset_index().fillna(0)
    proc_stoc = proc_stoc[proc_stoc["facility_id"].isin(facility_list)]
    item_types = pd.unique(demand_grouped.item_type)
    res = []
    proc_stoc_due_map = pd.DataFrame(
        columns=["item_type", "facility_id", "order_quantity"]
    )
    for item in item_types:
        local_demand_grouped = demand_grouped[demand_grouped["item_type"] == item]
        local_proc_stoc = proc_stoc[proc_stoc["item_type"] == item]
        temp_list = [0] * (demand_len + proc_stoc_len)
        gy_stock = [0, 0, 0]
        gy_exh = [False, False, False]
        exh_index = [99, 99, 99]
        # gy_stock = [0, 0]
        # gy_exh = [False, False]
        # exh_index = [99, 99]
        GY4 = 0
        for row in local_proc_stoc.itertuples():
            index = facility_list.index(row.facility_id) + demand_len
            local_processing = processing[
                (processing["item_type"] == item)
                & (processing["facility_id"] == row.facility_id)
            ]
            ### GY4_stock for GK/SET seperation
            additional_stock = None
            ### GY stocks

            if "GY" in row.facility_id:
                if row.facility_id in "GY2":
                    proc_sum = sum(
                        [
                            obj.get("gy").get("qty") if obj != 0 else 0
                            for obj in temp_list[index - 2 : index]
                        ]
                    )
                    gy_stock[0] = row.order_quantity + proc_sum
                elif row.facility_id == "GY3":
                    proc_sum = sum(
                        [
                            obj.get("gy").get("qty") if obj != 0 else 0
                            for obj in temp_list[index - 3 : index]
                        ]
                    )
                    gy_stock[1] = row.order_quantity + proc_sum
                elif row.facility_id == "GY4":
                    proc_sum = sum(
                        [
                            obj.get("gy").get("qty") if obj != 0 else 0
                            for obj in temp_list[index - 4 : index]
                        ]
                    )
                    gy_stock[2] = int(row.order_quantity[0]) + proc_sum
                    GY4 = row.order_quantity[0]
                    additional_stock = row.order_quantity[1]
                temp_list[index] = {
                    "additional": additional_stock,
                    "qty": row.order_quantity
                    if row.facility_id != "GY4"
                    else int(row.order_quantity[0]),
                    "proc_sum": proc_sum,
                    "type": "STOCK",
                }
            ### block processing
            else:
                temp_list[index] = {
                    "gy": {
                        "df": local_processing[local_processing["gy"] == "GY"].to_dict(
                            orient="records"
                        ),
                        "qty": row.gy_qty,
                    },
                    "hk": {
                        "df": local_processing[local_processing["gy"] == "HK"].to_dict(
                            orient="records"
                        ),
                        "qty": row.hk_qty,
                    },
                    "type": "PROCESSING",
                }

        local_proc_stoc = local_proc_stoc.sort_values(
            "facility_id", key=lambda x: x.map(facility_dict), ascending=False
        )
        local_proc_stoc.loc[
            local_proc_stoc["facility_id"] == "GY4", "order_quantity"
        ] = GY4
        local_proc_stoc["order_quantity"] = local_proc_stoc["order_quantity"].astype(
            float
        )
        local_proc_stoc["order_quantity"] = (
            local_proc_stoc["order_quantity"] + local_proc_stoc["gy_qty"]
        )
        local_proc_stoc = local_proc_stoc[
            ["item_type", "facility_id", "order_quantity"]
        ]
        local_proc_stoc["due"] = None
        for row in local_demand_grouped.itertuples():
            qty = [row.gy2_qty, row.gy3_qty, row.gy4_qty]
            breakpoint_values = [False, False, False]
            local_demand = demand[
                (demand["item_type"] == item) & (demand["week"] == row.week)
            ]
            # print(local_demand)
            ### calculate for each GY
            week_date = mondays[row.week - 1]
            if row.gy4_qty > 0:
                demand_qty = row.gy4_qty
                for proc in local_proc_stoc.itertuples():
                    surplus = proc.order_quantity - demand_qty
                    if surplus < 0:
                        if local_proc_stoc.loc[proc[0], "due"] == None:
                            local_proc_stoc.loc[proc[0], "due"] = week_date
                        local_proc_stoc.loc[proc[0], "order_quantity"] = 0
                        demand_qty -= proc.order_quantity
                    else:
                        if (
                            "GY" not in proc.facility_id
                            and local_proc_stoc.loc[proc[0], "due"] == None
                        ):
                            local_proc_stoc.loc[proc[0], "due"] = week_date
                        local_proc_stoc.loc[proc[0], "order_quantity"] = surplus
                        break

            if row.gy3_qty > 0:
                demand_qty = row.gy3_qty
                gy3_proc_stock = local_proc_stoc[
                    local_proc_stoc["facility_id"].isin(facility_list[:7])
                ]
                for proc in gy3_proc_stock.itertuples():
                    surplus = proc.order_quantity - demand_qty
                    if surplus < 0:
                        if local_proc_stoc.loc[proc[0], "due"] == None:
                            local_proc_stoc.loc[proc[0], "due"] = week_date
                        local_proc_stoc.loc[proc[0], "order_quantity"] = 0
                        demand_qty -= proc.order_quantity
                    else:
                        if local_proc_stoc.loc[proc[0], "due"] == None:
                            local_proc_stoc.loc[proc[0], "due"] = week_date
                        local_proc_stoc.loc[proc[0], "order_quantity"] = surplus
                        break

            if row.gy2_qty > 0:
                demand_qty = row.gy2_qty
                gy2_proc_stock = local_proc_stoc[
                    local_proc_stoc["facility_id"].isin(facility_list[:3])
                ]
                for proc in gy2_proc_stock.itertuples():
                    surplus = proc.order_quantity - demand_qty
                    if surplus < 0:
                        if local_proc_stoc.loc[proc[0], "due"] == None:
                            local_proc_stoc.loc[proc[0], "due"] = week_date
                        local_proc_stoc.loc[proc[0], "order_quantity"] = 0
                        demand_qty -= proc.order_quantity
                    else:
                        if local_proc_stoc.loc[proc[0], "due"] == None:
                            local_proc_stoc.loc[proc[0], "due"] = week_date
                        local_proc_stoc.loc[proc[0], "order_quantity"] = surplus
                        break

            for i, gy in enumerate(["GY2", "GY3", "GY4"]):
                gy_stock[i] -= qty[i]
                ### stock is negative
                if gy_stock[i] < 0:
                    ### stock gets negative
                    if gy_exh[i] == False:
                        ### exhaustion flag turn on
                        gy_exh[i] = True
                        ### exhaustion week record
                        exh_index[i] = row.week
                        covered_qty = qty[i] + gy_stock[i]
                        if covered_qty != 0:
                            breakpoint_values[i] = [covered_qty, -gy_stock[i]]
                            ### check which demand mos are covered. prioritize set order to block GK orders.
                            for local_demand_row in local_demand[
                                local_demand["hk_flag"] == "RBS"
                            ].itertuples():
                                covered_qty -= local_demand_row.qty
                                if covered_qty >= 0:
                                    demand.loc[
                                        local_demand_row[0], "block_covered"
                                    ] = True
                                else:
                                    break
                ### stock remains positive then block_covered true
                else:
                    demand.loc[
                        (demand["item_type"] == item)
                        & (demand["week"] == row.week)
                        & (demand["gy"] == gy),
                        "block_covered",
                    ] = True

            temp_list[row.week - 1] = {
                "gy2": {
                    "df": local_demand[local_demand["gy"] == "GY2"].to_dict(
                        orient="records"
                    ),
                    "qty": row.gy2_qty,
                    "exh": gy_exh[0],
                    "breakingpoint": breakpoint_values[0],
                },
                "gy3": {
                    "df": local_demand[local_demand["gy"] == "GY3"].to_dict(
                        orient="records"
                    ),
                    "qty": row.gy3_qty,
                    "exh": gy_exh[1],
                    "breakingpoint": breakpoint_values[1],
                },
                "gy4": {
                    "df": local_demand[local_demand["gy"] == "GY4"].to_dict(
                        orient="records"
                    ),
                    "qty": row.gy4_qty,
                    "exh": gy_exh[2],
                    "breakingpoint": breakpoint_values[2],
                },
                "type": "DEMAND",
            }
        proc_stoc_due_map = pd.concat([proc_stoc_due_map, local_proc_stoc])
        item_dict = {"item_name": item, "data": temp_list, "exh_index": exh_index}
        res.append(item_dict)
    block_coverage.set_data(demand)
    block_processing_write_databse(processing, proc_stoc_due_map)
    return {"res": res, "mondays": [i.strftime("%m-%d") for i in mondays]}


def get_block_kanban(update=False):
    if block_kanban.empty or update:
        block_kanban.set_data(generate_block_kanban())
    return block_kanban.get_data()





743    HSR45-5000L(GK) RAIL
911    HSR45-3000L(GK) RAIL
912    HSR45-3000L(GK) RAIL
Name: ItemName, dtype: object

In [29]:
def block_demand(force_update=False):
    today = date.today()
    query_day = 168 - timedelta(today.weekday()).days
    sql = f"""
    SELECT product_family, product_model, item_description, product_variant, order_scheduled_due, printed_due, SUM(order_quantity) AS qty, ARRAY_AGG(manufacturing_orders.order_number) as MOS, item_number, reference_number, facility_id
        FROM manufacturing_orders
        INNER join manufacturing_order_processes on manufacturing_orders.order_number = manufacturing_order_processes.order_number
        WHERE order_status ='10'
            AND order_release_code = 5
            AND facility_id in  ('A0010', 'A0045', 'E0010', 'J0005')
            AND product_model IS NOT NULL
            AND reference_number IS NOT NULL
            AND item_description like '%BLOCK%'
            AND order_scheduled_due <= CURRENT_DATE + {query_day}
        GROUP BY (product_family, product_model, product_variant, item_description, item_number, printed_due, reference_number, order_scheduled_due, printed_due, facility_id)    
    """
    return call_db_json(sql)


def check_date_group(dt, mondays):
    index = bisect.bisect_right(mondays, dt)
    if index == len(mondays):
        return 25
    else:
        return index


# grouped_df
def get_name(row):
    adder = "X" if row.product_family == "HSR" else ""
    return (
        row["product_family"]
        + row["product_model"]
        + adder
        + row["product_variant"]
        + row["is_M"]
    )


def get_block_process():
    block_proc_sql = """
    SELECT
        order_number,
        product_family,
        product_model,
        product_variant,
        item_description,    
        order_quantity,
        printed_due,
        item_number,
        (
        SELECT
            string_agg(facility_id, ' ')
        FROM manufacturing_order_processes
        WHERE manufacturing_order_processes.order_number = manufacturing_orders.order_number
        ) facility,
        (SELECT
            facility_id
        FROM manufacturing_order_processes
        WHERE operation_status != '40'
            AND manufacturing_order_processes.order_number = manufacturing_orders.order_number
        ORDER BY operation_sequence
        LIMIT 1) facility_id,
        reference_number,
        order_scheduled_due,
        (SELECT
            time_in
        FROM manufacturing_order_logs
        WHERE manufacturing_order_logs.order_number = manufacturing_orders.order_number
        ORDER BY time_in DESC
        LIMIT 1) time_out
        FROM manufacturing_orders
        WHERE manufacturing_orders.order_status ='40'
            AND manufacturing_orders.order_release_code = 5
            AND product_model IS NOT NULL
            AND product_family IS NOT NULL
            AND product_variant IS NOT NULL
            AND item_description NOT LIKE '%+%'
            AND (reference_number LIKE 'HK%' or reference_number IN ('GY2', 'GY3', 'GY4'))
            AND product_block_count > 0
            AND product_family != 'TS'
        ORDER BY order_number
    """
    proc_db = call_db_json(block_proc_sql)
    pdf = pd.DataFrame(proc_db)
    pdf["is_M"] = pdf["item_description"].apply(lambda x: "_M" if " M " in x else "")
    pdf["item_type"] = pdf.apply(get_name, axis=1)
    pdf["gy"] = pdf["reference_number"].apply(lambda x: "HK" if "HK" in x else "GY")
    rename = {"order_quantity": "qty", "order_number": "mos"}
    pdf = pdf.rename(columns=rename)
    p_groupby = pdf.groupby(["item_type", "facility_id", "gy"])
    p_grouped_df = p_groupby["qty"].sum()
    p_grouped_df = p_grouped_df.reset_index().set_index("item_type")
    p_grouped_df = p_grouped_df.pivot_table(
        index=["item_type", "facility_id"], columns="gy", values="qty", aggfunc="sum"
    ).reset_index()
    p_grouped_df.columns = ["item_type", "facility_id", "gy_qty", "hk_qty"]
    p_grouped_df["gy_qty"].fillna(0, inplace=True)
    p_grouped_df["hk_qty"].fillna(0, inplace=True)
    pdf = pdf[
        [
            "item_description",
            "order_scheduled_due",
            "qty",
            "reference_number",
            "mos",
            "item_type",
            "facility_id",
            "gy",
        ]
    ]
    return p_grouped_df, pdf


def get_block_stock():
    block_stock_sql = """
        SELECT item_number, item_description, product_family, product_model, product_variant, product_block_count, warehouse_location, quantity, virtual_location
            FROM current_wip_inventory
            WHERE block=true
                AND product_variant IS NOT NULL
                AND warehouse_location != 'INTRAN'
        """
    s_df = pd.DataFrame(call_db_json(block_stock_sql))
    s_df.head()
    s_df["is_M"] = s_df["item_description"].apply(lambda x: "_M" if " M " in x else "")
    s_df["item_type"] = s_df.apply(get_name, axis=1)
    s_df["virtual_location"] = s_df["virtual_location"].apply(
        lambda x: "GY1" if x == "BLOCK" else x
    )

    s_df["warehouse_location"] = s_df["warehouse_location"].apply(
        lambda x: "SET" if "SET" in x else ""
    )

    # s_df.loc[s_df['item_description'].str.contains('HOLED BLOCK 2'), 'virtual_location'] = 'GY2.5'
    s_groupby = s_df.groupby(["item_type", "virtual_location", "warehouse_location"])
    qty = s_groupby["quantity"].sum().reset_index()
    s_df = qty.pivot_table(
        index=["item_type"],
        columns=["warehouse_location", "virtual_location"],
        values="quantity",
    )
    s_df = s_df.fillna(0).reset_index()
    s_df.columns = ["item_type", "GY1", "GY2", "GY3", "GY4_gk", "GY4_set"]
    s_df[["GY4_gk", "GY4_set"]] = s_df[["GY4_gk", "GY4_set"]].astype(int)

    def gy4_concat(row):
        qty = row["GY4_gk"] + row["GY4_set"]
        combined_string = str(row["GY4_gk"]) + "/" + str(row["GY4_set"])
        return np.array([qty, combined_string])

    s_df["GY4"] = s_df.apply(gy4_concat, axis=1)
    s_df = s_df.drop(columns=["GY4_gk", "GY4_set"])
    s_df = pd.melt(
        s_df, id_vars="item_type", var_name="facility_id", value_name="order_quantity"
    )
    return s_df

def gy_checker(facility_id):
    if facility_id == "J0005":
        return 'GY4'
    elif facility_id == "E0010":
        return 'GY3'
    else:
        return 'GY2'


def get_block_demand():
    mondays = get_mondays(23)
    mondays = [i.date() for i in mondays]

    demand = pd.DataFrame(block_demand())
    try:
        demand["hk_flag"] = demand["reference_number"].apply(lambda x: hk_flag.flag[x])
    except:
        get_set_flag(True)
        demand["hk_flag"] = demand["reference_number"].apply(lambda x: hk_flag.flag[x])
    demand["order_scheduled_due"] = pd.to_datetime(demand["order_scheduled_due"])

    demand["week"] = demand["order_scheduled_due"].apply(
        check_date_group, args=[mondays]
    )
    demand["is_M"] = demand["item_description"].apply(
        lambda x: "_M" if " M " in x else ""
    )
    demand = demand.rename(columns={"facility_id": "gy"})

    demand["gy"] = demand["gy"].apply(gy_checker)
    demand["item_type"] = demand.apply(get_name, axis=1)
    d_groupby = demand.groupby(
        ["product_family", "product_model", "product_variant", "is_M", "week", "gy"]
    )

    qty = d_groupby["qty"].sum()
    demand_grouped = qty.reset_index()
    demand_grouped["item_type"] = demand_grouped.apply(get_name, axis=1)
    demand_grouped = demand_grouped.drop(
        columns=["product_family", "product_model", "product_variant", "is_M"]
    )
    demand_grouped = demand_grouped.pivot_table(
        index=["item_type", "week"], columns="gy", values="qty", aggfunc="sum"
    ).reset_index()
    demand_grouped.columns = ["item_type", "week", "gy2_qty", "gy3_qty", "gy4_qty"]
    # demand_grouped[["gy2_qty", "gy3_qty","gy4_qty"]].fillna(0, inplace=True)
    # demand_grouped["gy3_qty"].fillna(0, inplace=True)
    # demand_grouped["gy4_qty"].fillna(0, inplace=True)
    demand_grouped.fillna(0, inplace=True)
    demand = demand[
        [
            "item_description",
            "order_scheduled_due",
            "qty",
            "reference_number",
            "mos",
            "item_type",
            "week",
            "hk_flag",
            "gy",
        ]
    ].copy()
    demand['block_covered'] = False
    demand = demand.sort_values('order_scheduled_due')
    return demand_grouped, demand, mondays


def block_processing_write_databse(processing, proc_stoc_due_map):
    processing = processing.merge(
        proc_stoc_due_map[["item_type", "facility_id", "due"]],
        on=["item_type", "facility_id"],
        how="left",
    )
    processing.loc[
        processing["order_scheduled_due"].isnull(), "order_scheduled_due"
    ] = processing["due"]
    processing = processing.fillna("")
    processing["order_scheduled_due"] = processing["order_scheduled_due"].apply(
        lambda x: x.strftime("%Y-%m-%d") if x != "" else None
    )
    processing = (
        processing.drop(columns=["gy", "due"])
        .rename(columns={"mos": "mo"})
        .fillna("NO DUE")
    )
    processing = processing.groupby(["facility_id"]).apply(
        lambda x: x.sort_values("order_scheduled_due")
    )
    processing.reset_index(drop=True, inplace=True)
    processing["facility_priority"] = processing.groupby(["facility_id"]).cumcount() + 1
    table = "ignition.initial_release.block_priority"
    delete_sql = f"TRUNCATE {table}; DELETE FROM {table}"
    update_ignition(delete_sql)
    insert_many_with_df("ignition", processing, "initial_release.block_priority")


def generate_block_kanban():
    facility_list = [
        "A0020",
        "A0030",
        "GY2",
        "A0050",
        "GY2.5",
        "A0070",
        "GY3",
        "E0020",
        "F0020",
        "E0030",
        "E0050",
        "GY4",
    ]
    facility_dict = {}
    for i, v in enumerate(facility_list):
        facility_dict[v] = i
    demand_len = 25
    proc_stoc_len = len(facility_list)
    demand_grouped, demand, mondays = get_block_demand()    
    processing_grouped, processing = get_block_process()
    stock_df = get_block_stock()
    proc_stoc = pd.concat([processing_grouped, stock_df]).reset_index().fillna(0)
    proc_stoc = proc_stoc[proc_stoc["facility_id"].isin(facility_list)]
    item_types = pd.unique(demand_grouped.item_type)
    res = []
    proc_stoc_due_map = pd.DataFrame(
        columns=["item_type", "facility_id", "order_quantity"]
    )
    for item in item_types:
        local_demand_grouped = demand_grouped[demand_grouped["item_type"] == item]
        local_proc_stoc = proc_stoc[proc_stoc["item_type"] == item]
        temp_list = [0] * (demand_len + proc_stoc_len)        
        gy_stock = [0, 0, 0]
        gy_exh = [False, False, False]
        exh_index = [99, 99, 99]
        # gy_stock = [0, 0]
        # gy_exh = [False, False]
        # exh_index = [99, 99]
        GY4 = 0
        for row in local_proc_stoc.itertuples():
            index = facility_list.index(row.facility_id) + demand_len
            local_processing = processing[
                (processing["item_type"] == item)
                & (processing["facility_id"] == row.facility_id)
            ]
            ### GY4_stock for GK/SET seperation
            additional_stock = None
            ### GY stocks

            if "GY" in row.facility_id:
                if row.facility_id == "GY2":
                    proc_sum = sum(
                        [
                            obj.get("gy").get("qty") if obj != 0 else 0
                            for obj in temp_list[index - 2 : index]
                        ]
                    )
                    gy_stock[0] = row.order_quantity + proc_sum
                elif row.facility_id == "GY3":
                    proc_sum = sum(
                        [
                            obj.get("gy").get("qty") if obj != 0 else 0
                            for obj in temp_list[index - 3 : index]
                        ]
                    )
                    gy_stock[1] = row.order_quantity + proc_sum
                elif row.facility_id == "GY4":
                    proc_sum = sum(
                        [
                            obj.get("gy").get("qty") if obj != 0 else 0
                            for obj in temp_list[index - 4 : index]
                        ]
                    )
                    gy_stock[2] = int(row.order_quantity[0]) + proc_sum
                    GY4 = row.order_quantity[0]
                    additional_stock = row.order_quantity[1]
                temp_list[index] = {
                    "additional": additional_stock,
                    "qty": row.order_quantity
                    if row.facility_id != "GY4"
                    else int(row.order_quantity[0]),
                    "proc_sum": proc_sum,
                    "type": "STOCK",
                }
            ### block processing
            else:
                temp_list[index] = {
                    "gy": {
                        "df": local_processing[local_processing["gy"] == "GY"].to_dict(
                            orient="records"
                        ),
                        "qty": row.gy_qty,
                    },
                    "hk": {
                        "df": local_processing[local_processing["gy"] == "HK"].to_dict(
                            orient="records"
                        ),
                        "qty": row.hk_qty,
                    },
                    "type": "PROCESSING",
                }

        local_proc_stoc = local_proc_stoc.sort_values(
            "facility_id", key=lambda x: x.map(facility_dict), ascending=False
        )
        local_proc_stoc.loc[
            local_proc_stoc["facility_id"] == "GY4", "order_quantity"
        ] = GY4
        local_proc_stoc["order_quantity"] = local_proc_stoc["order_quantity"].astype(
            float
        )
        local_proc_stoc["order_quantity"] = (
            local_proc_stoc["order_quantity"] + local_proc_stoc["gy_qty"]
        )
        local_proc_stoc = local_proc_stoc[
            ["item_type", "facility_id", "order_quantity"]
        ]
        local_proc_stoc["due"] = None
        for row in local_demand_grouped.itertuples():
            qty = [row.gy2_qty, row.gy3_qty, row.gy4_qty]
            breakpoint_values = [False, False, False]
            local_demand = demand[
                (demand["item_type"] == item) & (demand["week"] == row.week)
            ]
            # print(local_demand)
            ### calculate for each GY
            week_date = mondays[row.week - 1]
            if row.gy4_qty > 0:
                demand_qty = row.gy4_qty
                for proc in local_proc_stoc.itertuples():
                    surplus = proc.order_quantity - demand_qty
                    if surplus < 0:
                        if local_proc_stoc.loc[proc[0], "due"] == None:
                            local_proc_stoc.loc[proc[0], "due"] = week_date
                        local_proc_stoc.loc[proc[0], "order_quantity"] = 0
                        demand_qty -= proc.order_quantity
                    else:
                        if (
                            "GY" not in proc.facility_id
                            and local_proc_stoc.loc[proc[0], "due"] == None
                        ):
                            local_proc_stoc.loc[proc[0], "due"] = week_date
                        local_proc_stoc.loc[proc[0], "order_quantity"] = surplus
                        break

            if row.gy3_qty > 0:
                demand_qty = row.gy3_qty                
                gy3_proc_stock = local_proc_stoc[
                    local_proc_stoc["facility_id"].isin(facility_list[:7])
                ]
                for proc in gy3_proc_stock.itertuples():
                    surplus = proc.order_quantity - demand_qty
                    if surplus < 0:
                        if local_proc_stoc.loc[proc[0], "due"] == None:
                            local_proc_stoc.loc[proc[0], "due"] = week_date
                        local_proc_stoc.loc[proc[0], "order_quantity"] = 0
                        demand_qty -= proc.order_quantity
                    else:
                        if local_proc_stoc.loc[proc[0], "due"] == None:
                            local_proc_stoc.loc[proc[0], "due"] = week_date
                        local_proc_stoc.loc[proc[0], "order_quantity"] = surplus
                        break

            if row.gy2_qty > 0:
                demand_qty = row.gy2_qty                
                gy2_proc_stock = local_proc_stoc[
                    local_proc_stoc["facility_id"].isin(facility_list[:3])
                ]
                for proc in gy2_proc_stock.itertuples():
                    surplus = proc.order_quantity - demand_qty
                    if surplus < 0:
                        if local_proc_stoc.loc[proc[0], "due"] == None:
                            local_proc_stoc.loc[proc[0], "due"] = week_date
                        local_proc_stoc.loc[proc[0], "order_quantity"] = 0
                        demand_qty -= proc.order_quantity
                    else:
                        if local_proc_stoc.loc[proc[0], "due"] == None:
                            local_proc_stoc.loc[proc[0], "due"] = week_date
                        local_proc_stoc.loc[proc[0], "order_quantity"] = surplus
                        break
            
            for i, gy in enumerate(['GY2', 'GY3', 'GY4']):
                gy_stock[i] -= qty[i]                
                ### stock is negative
                if gy_stock[i] < 0:
                    ### stock gets negative
                    if gy_exh[i] == False:                    
                        ### exhaustion flag turn on
                        gy_exh[i] = True
                        ### exhaustion week record
                        exh_index[i] = row.week
                        covered_qty = qty[i] + gy_stock[i]
                        if covered_qty != 0:                            
                            breakpoint_values[i] = [covered_qty, -gy_stock[i]]
                            ### check which demand mos are covered. prioritize set order to block GK orders.
                            for local_demand_row in local_demand[local_demand['hk_flag'] == 'RBS'].itertuples():
                                covered_qty -= local_demand_row.qty
                                if covered_qty >= 0:
                                    demand.loc[local_demand_row[0], 'block_covered'] = True
                                else:
                                    break
                ### stock remains positive then block_covered true
                else:
                    demand.loc[(demand["item_type"] == item) & (demand["week"] == row.week) & (demand['gy'] == gy), 'block_covered'] = True                                

            temp_list[row.week - 1] = {
                "gy2": {
                    "df": local_demand[local_demand["gy"] == "GY2"].to_dict(
                        orient="records"
                    ),
                    "qty": row.gy2_qty,
                    "exh": gy_exh[0],
                    "breakingpoint": breakpoint_values[0],
                },
                "gy3": {
                    "df": local_demand[local_demand["gy"] == "GY3"].to_dict(
                        orient="records"
                    ),
                    "qty": row.gy3_qty,
                    "exh": gy_exh[1],
                    "breakingpoint": breakpoint_values[1],
                },
                "gy4": {
                    "df": local_demand[local_demand["gy"] == "GY4"].to_dict(
                        orient="records"
                    ),
                    "qty": row.gy4_qty,
                    "exh": gy_exh[2],
                    "breakingpoint": breakpoint_values[2],
                },
                "type": "DEMAND",
            }
        proc_stoc_due_map = pd.concat([proc_stoc_due_map, local_proc_stoc])
        item_dict = {"item_name": item, "data": temp_list, "exh_index": exh_index}
        res.append(item_dict)
    block_coverage.set_data(demand)
    block_processing_write_databse(processing, proc_stoc_due_map)
    return {"res": res, "mondays": [i.strftime("%m-%d") for i in mondays]}, demand


def get_block_kanban(update=False):
    if block_kanban.empty or update:
        block_kanban.set_data(generate_block_kanban())
    return block_kanban.get_data()


In [30]:
a, df = generate_block_kanban()

C:\Users\Ejeong\AppData\Local\Temp\ipykernel_7588\2194596814.py:21: FutureWarning: Comparison of Timestamp with datetime.date is deprecated in order to match the standard library behavior. In a future version these will be considered non-comparable. Use 'ts == pd.Timestamp(date)' or 'ts.date() == date' instead.
  index = bisect.bisect_right(mondays, dt)


execute_many() done


In [67]:
def check_stock_type(row):
    if "SET" in row.warehouse_location:
        return "SET"
    elif "HOLED" in row.item_description:
        return "HOLED"
    else:
        return ""


def get_block_stock():
    block_stock_sql = """
        SELECT item_number, item_description, product_family, product_model, product_variant, product_block_count, warehouse_location, quantity, virtual_location
            FROM current_wip_inventory
            WHERE block=true
                AND product_variant IS NOT NULL
                AND warehouse_location != 'INTRAN'
        """
    s_df = pd.DataFrame(call_db_json(block_stock_sql))
    s_df.head()
    s_df["is_M"] = s_df["item_description"].apply(lambda x: "_M" if " M " in x else "")
    s_df["item_type"] = s_df.apply(get_name, axis=1)
    s_df["virtual_location"] = s_df["virtual_location"].apply(
        lambda x: "GY1" if x == "BLOCK" else x
    )


    # s_df["warehouse_location"] = s_df["warehouse_location"].apply(
    #     lambda x: "SET" if "SET" in x else ""
    # )
    s_df["warehouse_location"] = s_df.apply(check_stock_type, axis=1)    

    # s_df.loc[s_df['item_description'].str.contains('HOLED BLOCK'), 'virtual_location'] = 'GY2.5'
    s_groupby = s_df.groupby(["item_type", "virtual_location", "warehouse_location"])
    qty = s_groupby["quantity"].sum().reset_index()
    s_df = qty.pivot_table(
        index=["item_type"],
        columns=["warehouse_location", "virtual_location"],
        values="quantity",
    )
    s_df = s_df.fillna(0).reset_index()
    s_df.columns = ["item_type", "GY1", "GY2", "GY3", "GY4_gk", "GY2_HOLED", "GY4_set"]
    # s_df[["GY4_gk", "GY4_set"]] = s_df[["GY4_gk", "GY4_set"]].astype(int)
    # s_df[["GY4_gk", "GY4_set"]] = s_df[["GY4_gk", "GY4_set"]].astype(int)

    def gy4_concat(row):
        qty = row["GY4_gk"] + row["GY4_set"]
        combined_string = str(row["GY4_gk"]) + "/" + str(row["GY4_set"])
        return np.array([qty, combined_string])
        

    s_df["GY4"] = s_df.apply(gy4_concat, axis=1)
    s_df["GY2"] = s_df.apply(gy4_concat, axis=1)
    # s_df = s_df.drop(columns=["GY4_gk", "GY4_set"])
    # s_df = pd.melt(
    #     s_df, id_vars="item_type", var_name="facility_id", value_name="order_quantity"
    # )
    return s_df
a = get_block_stock()

In [70]:
a[a['facility_id'] == 'GY2']

,item_type,facility_id,order_quantity
247,HRW17CA,GY2,"[0.0, 0.0/0.0]"
248,HRW17CA_M,GY2,"[0.0, 0.0/0.0]"
249,HRW17CR,GY2,"[0.0, 0.0/0.0]"
250,HRW17CR_M,GY2,"[0.0, 0.0/0.0]"
251,HRW21CA,GY2,"[0.0, 0.0/0.0]"
...,...,...,...
489,SSR25XW,GY2,"[0.0, 0.0/0.0]"
490,SSR25XW_M,GY2,"[0.0, 0.0/0.0]"
491,SSR30XW,GY2,"[0.0, 0.0/0.0]"
492,SSR30XW_M,GY2,"[0.0, 0.0/0.0]"


In [ ]:
# import os
# KANBAN_PATH = (
#     "N:\HeatTreat\Rail Kanban\RAIL KANBAN Ver6.05.xlsm"
#     if os.name == "nt"
#     else "/mnt/windows/HeatTreat/Rail Kanban/RAIL KANBAN Ver6.05.xlsm"
# )
# kanban_base = pd.ExcelFile(KANBAN_PATH)
# kanban = pd.read_excel(kanban_base, skiprows=4, nrows=141)
# df = pd.read_excel(kanban_base, "Juchu Data")
# test = df[df['BlockType.1'].str.contains('HSR45')].copy()
# t = pd.unique(test['BlockType.1'])
# df[df['BlockType.1'] == 'HSR45X']['ItemName']
# t